In [ ]:
# import the required libraries
import docx2txt
import os
import re
import csv
import pandas as pd
import numpy as np
import act1

In [ ]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = '..//data//'
input_folder = 'input//'
input_file = os.path.join(folderlink+input_folder, 'Repertoriummetoudepersoonsnummers.docx')
folder_output = 'output//'
output_txt = folderlink+folder_output+'output.txt'
output_csv = folderlink+folder_output+'output_file.csv'

In [ ]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# Converting the docx to a text file and remove all unecessary rows.

# Use docx2txt library to extract text from .docx file
text = docx2txt.process(input_file)

# Remove excessive whitespaces
data = ' '.join(text.split())

Manual updates in the dataset:

1.  FROM: "N.N. "de oude vicarius">pred. Lichtenvoorde 1602 tot 1615.<20871>"
    TO: "N.N. "de oude vicarius" pred. Lichtenvoorde 1602 tot 1615.<20871>" 
    REASON: since it contained the > character which is used to define ID fields. 
 
2.  FROM: "Bosch; Cornelius Geb. Utrecht 1634; pred. Renswoude 16 dec. 1656, Maasland 15 april 1663, Brielle 30 jan. 1667, Alkmaar 1667, 's Gravenhage 5 juli 1676, emer. 1713,;overl. 28 maart 1715.<1185>"
    TO:Bosch; Cornelius
Geb. Utrecht 1634; pred. Renswoude 16 dec. 1656, Maasland 15 april 1663, Brielle 30 jan. 1667, Alkmaar 1667, 's Gravenhage 5 juli 1676, emer. 1713, overl. 28 maart 1715.<1185>
    REASON: Since it splitted the string based on ",;overl"

3.  FROM: Leeuwen, van, Cornelis [z.v. Cornelis]
Geb. Hazerswoude 1611; pred. Boskoop en Middelburg 1637, overl. 1681.<5778>
    TO: Leeuwen, van; Cornelis [z.v. Cornelis]
Geb. Hazerswoude 1611; pred. Boskoop en Middelburg 1637, overl. 1681.<5778>
    REASON: there was no ; between the name and surname (including infix)

4.  FROM: Peenen, van, Marcus
Gedoopt Leiden 31 aug. 1642; pred. Koudekerk aan den Rijn 2 sept. 1668, Leiden 1680, begraven 1 febr. 1696.<7388>
    TO: Peenen, van; Marcus
Gedoopt Leiden 31 aug. 1642; pred. Koudekerk aan den Rijn 2 sept. 1668, Leiden 1680, begraven 1 febr. 1696.<7388>
    REASON: there was no ; between the name and surname (including infix)

5.  FROM: Knuyt, de (Kuntius), Elias
Geb. Middelburg yyy; pred. Oude Niedorp en Veenhuizen (NH) mei 1628, Sint Annaland 1630, Westkapelle 7 maart 1641, overl. --of vertrokken?-- 1642.<5381>
    TO: Knuyt, de (Kuntius); Elias
Geb. Middelburg yyy; pred. Oude Niedorp en Veenhuizen (NH) mei 1628, Sint Annaland 1630, Westkapelle 7 maart 1641, overl. --of vertrokken?-- 1642.<5381>
    REASON: there was no ; between the name and surname (including infix)

6.  FROM: Leonardis, de, Paulus
Geb. Keulen yyy; pred. Bacharach (Pfalz) 16.., Kampen 1620, overl. 1649.<5836>
    TO: Leonardis, de; Paulus
Geb. Keulen yyy; pred. Bacharach (Pfalz) 16.., Kampen 1620, overl. 1649.<5836>
    REASON: there was no ; between the name and surname (including infix)

7.  FROM: Tamerus; Henricus
Geb. xxx ca. 1540; voorheen Lutheraan;--onwettig pred. Heteren en Randwijk ca. 1600-1602; pred. Eethen en Meeuwen 1606, Doeveren, Gansoyen en Genderen 1610, afgezet als remonstrant 1619; overl. Heusden.<21206>
    TO: Tamerus; Henricus
Geb. xxx ca. 1540; voorheen Lutheraan; onwettig pred. Heteren en Randwijk ca. 1600-1602; pred. Eethen en Meeuwen 1606, Doeveren, Gansoyen en Genderen 1610, afgezet als remonstrant 1619; overl. Heusden.<21206>
    REASON: -- conflicted created a line break. 




In [ ]:
data = data.replace('overl. 28 maart 1715.<1185>', ' overl. 28 maart 1715.<1185>').replace('>pred.', ' pred.').replace('Leeuwen, van, Cornelis [z.v. Cornelis]','Leeuwen, van; Cornelis [z.v. Cornelis]',).replace('Peenen, van, Marcus','Peenen, van; Marcus').replace('Knuyt, de (Kuntius), Elias','Knuyt, de (Kuntius); Elias').replace('Leonardis, de, Paulus','Leonardis, de; Paulus').replace('Lutheraan;--onwettig','Lutheraan; onwettig')

In [ ]:
# Replace semicolons with newlines and add semicolons around < and > since these identify the IDs
data = data.replace(';', ';\n').replace(';\n ', '; ').replace('>', '>;\n ').replace('<', ';<')
lines = data.split('\n')

lines = [line for line in lines if not line.startswith('; ;') and not line.startswith('; ')]
data = '\n'.join(lines)
lines = data.strip().split('\n')
data = '\n'.join([line.lstrip() for line in lines])


In [ ]:
lines = data.split('\n')

with open(output_txt, "w", encoding='utf-8') as file:
    for line in lines:
        result_string = act1.replace_pred_count(line)
        file.write(result_string + "\n")

In [ ]:
columns = ("Geb.", # deze is cruciaal
 "0 pred.",
 "1 pred.",
 "2 pred.",
 "3 pred.",
 "4 pred.",
 "overl.", # deze is cruciaal
 "Gedoopt", # deze is cruciaal
 "legerpred.",
 "pastoor",
 "garnizoenspred.",
 "emer.",
 "begraven",
 "conrector",
 " rector",
 "monnik",           
 "schoolmeester",
 "hoogleraar",
 "chirurgijn",
 "praeceptor",
 "ziekentrooster",
 "vlootpred.",
 "legerpred.",
 "ambassadepred."           
)          
    

In [ ]:
for column in columns:
    with open(output_txt, 'r', encoding='utf-8') as f:
        lines = f.readlines()        
    with open(output_txt,'w', encoding='utf-8') as f:
        for line in lines:
            if "; "+column in line:
                f.write(line)
            elif column in line:
                line = line.replace(column, ";"+column)
                f.write(line)
            else:
                f.write(line)

In [ ]:
# Define the headers for the output file
headers = ['surname_temp', 'name', 'Field1', 'Field2', 'Field3', 'Field4', 'Field5', 'Field6', 'Field7', 'Field8', 'Field9', 'Field10', 'Field11','Field12','Field13','Field14','Field15','Field16','Field17']

with open(output_txt, 'r', encoding='utf-8') as infile, open(output_csv, 'w', newline='', encoding='utf-8' ) as outfile:
    reader = csv.reader(infile, delimiter=';')
    writer = csv.writer(outfile, delimiter=';')
    
    # Write the headers to the output file
    writer.writerow(headers)

    # Loop through each row in the input file and write it to the output file with 12 fields
    for row in reader:
        # Create a new row with 12 fields by extending the current row with empty values
        new_row = row + [''] * (12 - len(row))
        writer.writerow(new_row)

In [ ]:
df = pd.read_csv(output_csv, sep=';', encoding='utf-8')

In [ ]:
#In the file all IDs are stored between < and > therefore 
for column in df.columns:
    # Check if any value in the column contains '<'
    if df[column].astype(str).str.contains('<').any():
        # Copy the values containing '<' to the 'test' column
        df.loc[df[column].astype(str).str.contains('<'), 'clerus_id'] = df[column]


In [ ]:
df['clerus_id'] = df['clerus_id'].str.replace('>', '').str.replace('<','')

In [ ]:
for column in columns:
    df[column] = df.apply(lambda row: row[row.astype(str).str.contains(column)].iloc[0] if any(row.astype(str).str.contains(column)) else '', axis=1)

In [ ]:
df['original_input'] = df['surname_temp'].fillna('') + df['name'].fillna('') + df['Field1'].fillna('') + df['Field2'].fillna('')+ df['Field3'].fillna('')+ df['Field4'].fillna('')+ df['Field5'].fillna('')+ df['Field6'].fillna('')+ df['Field7'].fillna('')+ df['Field8'].fillna('')+ df['Field9'].fillna('')+ df['Field10'].fillna('')+ df['Field11'].fillna('')+ df['Field12'].fillna('')+ df['Field13'].fillna('')+ df['Field14'].fillna('')+ df['Field15'].fillna('')+ df['Field16'].fillna('')+ df['Field17'].fillna('')
df['original_input'] = df['original_input'].str.replace('0 pred. ', ' pred. ').str.replace('1  pred.',' pred. ').str.replace('2 pred.',' pred. ').str.replace('3  pred.',' pred. ').str.replace('4  pred.',' pred. ')

In [ ]:
for column in columns:
    df[column] = df[column].str.replace(column, '')

In [ ]:
df['name_info_family'] =df['name'] .str.extract(r'\[(.*?)\]')


In [ ]:
df['nickname'] =df['surname_temp'] .str.extract(r'\((.*?)\)')


In [ ]:
columns_rename = {
    'Geb.': 'birth', 
    #' pred.': 'minister', 
    'overl.': 'death', 
    'Gedoopt':'baptized', 
    'legerpred.':'legerpredikant',
    'pastoor':'pastoor',
    'garnizoenspred.':'garnizoenspredikant',
    "emer.":'emeritus_status',
    "begraven":'burried',
    "conrector":'conrector',
    " rector":'rector',
    "monnik":'monnik',           
    "schoolmeester":'schoolmeester',
    "hoogleraar":'hoogleraar',
    "chirurgijn":'chirurgijn',
    "praeceptor":'praeceptor',
    "ziekentrooster":'ziekentrooster',
    "vlootpred.":'vlootpredikant',
    "legerpred.":'legerpredikant',
    "ambassadepred.":'ambassadepredikant'}

In [ ]:
# Rename the columns
df = df.rename(columns=columns_rename)
new_columns = list(columns_rename.values())


In [ ]:
df.head()

In [ ]:
array_drop = [i for i in range(1, 18)]
for dropid in array_drop:
    column_dropid = 'Field'+str(dropid)
    df = df.drop(column_dropid, axis=1)

In [ ]:
def extract_year(text):
    match = re.search(r'\d{4}', text)
    if match:
        return match.group(0)
    else:
        return None

In [ ]:
df['minister'] = df['0 pred.']+ ','+df['1 pred.']+ ','+df['2 pred.']+ ','+df['3 pred.']+ ','+df['4 pred.']

In [ ]:
df = df.drop(['0 pred.','1 pred.','2 pred.','3 pred.','4 pred.'], axis=1)

In [ ]:
function_year = [word for word in new_columns if word != 'minister']

print(function_year)

In [ ]:
df.head()

In [ ]:
for year in function_year:
    fld_year = 'year_' +year
    df[fld_year] = df[year].apply(lambda x: extract_year(x))    

In [ ]:
for year_accu in function_year:
    accu_fld_year = 'accu_year_' +year_accu
    df[accu_fld_year] = ''
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Check if the string contains "ca." (case-insensitive)
        if 'ca.' in row[year_accu].lower():
            # If found, set the value of the "accuracy" column to "circa"
            df.at[index, accu_fld_year] = 'circa'
            
    

In [ ]:
months =(" januari ", 
 " februari ",
 " maart ",
 " april ",
 " mei ",
 " juni ",
 " juli ",
 " augustus ",
 " september ",
 " oktober ",
 " november ",           
 " december ",
 "jan. ",
 "feb. ",
 "mrt. ",
 "apr. ",
 "jun. ",
 "jul. ",
 "aug. ",
 "sept. ",
 "sep. ",
 "okt. ",
 "nov. ",
 "dec. ",
 "yyy",
 "xxx",
 "ca.",
 "febr."        
) 


In [ ]:
for column_strip in function_year:   
    for month in months:
        df[column_strip] = df[column_strip].str.replace(month, '')


In [ ]:
for column_strip in function_year:
    df[column_strip] = df[column_strip].apply(lambda x: re.sub(r'[\d\.]', '', x))

In [ ]:
df.head()

In [ ]:
df['surname_temp']= df['surname_temp'].str.replace(r'\(.*\)', '', regex=True)
df['name']= df['name'].str.replace(r'\[.*\]', '', regex=True)

In [ ]:
df.head()

In [ ]:
df[['surname', 'infix']] = df['surname_temp'].str.split(',', expand=True)

In [ ]:
df.head()

In [ ]:
df = df.drop(['surname_temp'], axis=1)

In [ ]:
df['join_name'] = df['surname']+df['name']+df['infix'].fillna('')
df['join_name'] = df['join_name'].str.replace("  "," ")

In [ ]:
df.head()

In [ ]:

birth_org = df[['clerus_id','birth','year_birth','accu_year_birth']]
birth_org.to_csv(folderlink+folder_output+'birth_info.csv', sep=';', encoding='utf-8', index=False)


In [ ]:
death_org = df[['clerus_id','death','year_death','accu_year_death']]
death_org.to_csv(folderlink+folder_output+'death_info.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
bapt_org = df[['clerus_id','baptized','year_baptized','accu_year_baptized']]
bapt_org.to_csv(folderlink+folder_output+'bapt_info.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
df.to_csv(folderlink+folder_output+'parent_data.csv', sep=';', encoding='utf-8', index=False)

From here we are going to make the parent child relationship with predikanten Ministers

Here the minister data is set as a child. 

In [ ]:
subset_pred = df[['clerus_id','minister','join_name','original_input']]

In [ ]:
subset_pred.head()

In [ ]:
df_expanded = subset_pred.assign(minister=subset_pred['minister'].str.split(','))

# Explode the 'pred.' column to create separate rows for each item
df_expanded = df_expanded.explode('minister')

In [ ]:
df_expanded.head()

In [ ]:
df_filtered = df_expanded[['clerus_id','minister','join_name','original_input']]

In [ ]:
childs = df_filtered[(df_filtered["minister"] !=" ") & (df_filtered["minister"] !="")]

In [ ]:
childs.head()

In [ ]:
childs['minister_year'] = childs['minister'].apply(lambda x: extract_year(x))

In [ ]:
accu_fld_year = 'accu_year_' +'minster'
childs[accu_fld_year] = ''
# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if the string contains "ca." (case-insensitive)
    if 'ca.' in row[year_accu].lower():
        # If found, set the value of the "accuracy" column to "circa"
        df.at[index, accu_fld_year] = 'circa'

In [ ]:
childs.head()

In [ ]:
for month in months:
        childs['minister'] = childs['minister'].str.replace(month, '')

childs['minister'] = childs['minister'].apply(lambda x: re.sub(r'[\d\.]', '', x))

In [ ]:
childs.head()

In [ ]:
childs.to_csv(folderlink+folder_output+'minister_info.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
os.remove(output_txt)
os.remove(output_csv)